In [2]:
import plotly.express as px
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import splrep, splprep, splev, BSpline
%matplotlib notebook

from pyubx2 import UBXReader

In [3]:
BASEDIR = "/home/delaynie/Documents/rtklib-workspace/cmubuggy-datalogger/data/ubx-files/"

In [4]:
def load_trace(fn):
    print(fn)
    msgs = []
    with tqdm(total=os.path.getsize(fn)) as pbar:
        with open(fn, "rb") as stream:
            offs = 0
            ubr = UBXReader(stream, protfilter=2)
            for x in tqdm(ubr):
                msgs.append(x[1])
                pbar.update(stream.tell() - offs)
                offs = stream.tell()

    print(f"# of messages: {len(msgs)}")
    print(set([x.identity for x in msgs]))

    out = []
    
    last_speed = 0
    last_time = 0
    
    for i, x in enumerate(msgs):
        print(type(x))
        if x.identity=="NAV-PVT" and x.hAcc < 10000 and i != 0:
            gspeed = round(x.gSpeed*0.00223694, 2) # mph
            ts = round(x.hour*60*60 + x.min*60 + x.second + x.nano*1e-9, 2)
            res = {
                "timestamp": ts,
                "timestamp_str": f"{x.year:04d}_{x.month:02d}_{x.day:02d}_{x.hour:02d}_{x.min:02d}_{x.second:02d}",
                "rtktype": x.carrSoln,
                "latitude": x.lat,
                "longitude": x.lon,
                "ground_speed": gspeed, # mph
                "acceleration": (gspeed - last_speed)/(ts - last_time) if last_time else 0,
                "altitude": round(x.hMSL / 1000, 2), # meters
                "horizontal_accuracy": round(x.hAcc / 1000, 2),
                "vertical_accuracy": round(x.vAcc / 1000, 2),
                "speed_accuracy": round(x.sAcc*0.00223694, 2),
                "msg_num": i
            }
            
            out.append(res)
            last_speed = gspeed
            last_time = ts

    df = pd.DataFrame.from_dict(out)
    print(f"{len(df)}, {df.keys()}")
    
    ll = list(df["timestamp"])
    oo = []
    for i in range(len(ll)-1):
        oo.append(round(ll[i+1]-ll[i], 2))

    print({x: oo.count(x) for x in set(oo)})
    
    return df
    

    
    

In [5]:
def plot_trace(df):
    layers = [{"below": "traces",
               "sourcetype": "raster",
               "source": ["https://imagery.pasda.psu.edu/arcgis/rest/services/pasda/PEMAImagery2018_2020/MapServer/WMTS/tile/1.0.0/pasda_PEMAImagery2018_2020/default/default028mm/{z}/{y}/{x}.png"]}]

    fig = px.scatter_mapbox(df,
                            lat="latitude", lon="longitude",
                            color="ground_speed",
                            hover_data=["horizontal_accuracy",
                                        "speed_accuracy",
                                        "altitude",
                                        "timestamp",
                                        "timestamp_str",
                                        "msg_num"],
                            zoom=15, size_max=18,
                            height=400, width=800)

    fig.update_layout(
        mapbox_style="white-bg",
        mapbox_layers=layers
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [6]:
RESOLUTION = 0.05

def prep_trace(df_in, idx=None):
    step = np.arange(df_in["timestamp"].min(), df_in["timestamp"].max(), 0.05)
    
    # fit a spline to the position
    pos_tck, u = splprep([df_in["latitude"].to_list(), df_in["longitude"].to_list()],
                  u=df_in["timestamp"].to_list(), s=0)
    
    x1, y1 = splev(step, pos_tck)

    vel_tck = splrep(df_in["timestamp"], df_in["ground_speed"], s=50)
    velo = BSpline(*vel_tck)(step, 0)
    accl = BSpline(*vel_tck)(step, 1)
    
    df = pd.DataFrame.from_dict({**{
        "timestamp": [round(x, 2) for x in step][:],
        "latitude": x1[:],
        "longitude": y1[:],
        "velocity": [round(x, 2) for x in velo][:],
        "acceleration": [round(x, 2) for x in accl][:],
        "time": [round(x-step[0], 2) for x in step][:]
    }, **({"roll_idx": [idx for _ in step]} if idx is not None else {})}, orient="index").transpose()
    
    return df

In [7]:
df = load_trace(BASEDIR+"scuwuter.ubx")

/home/delaynie/Documents/rtklib-workspace/cmubuggy-datalogger/data/ubx-files/scuwuter.ubx


  1%|          | 52936/6679628 [00:00<00:52, 126035.61it/s]

13241it [00:54, 244.72it/s]79628 [00:53<00:00, 106077.94it/s]
100%|██████████| 6679628/6679628 [00:54<00:00, 123406.64it/s]


# of messages: 13241
{'NAV-PVT', 'RXM-RAWX', 'RXM-SFRBX'}
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class 'pyubx2.ubxmessage.UBXMessage'>
<class

In [8]:
df

,timestamp,timestamp_str,rtktype,latitude,longitude,ground_speed,acceleration,altitude,horizontal_accuracy,vertical_accuracy,speed_accuracy,msg_num
0,64085.6,2023_03_24_17_48_05,0,40.449090,-79.951970,10.05,0.0,289.19,0.44,0.72,0.17,1
1,64085.7,2023_03_24_17_48_05,0,40.449087,-79.951968,9.94,-1.1,289.19,0.43,0.72,0.18,3
2,64085.8,2023_03_24_17_48_05,0,40.449083,-79.951966,9.74,-2.0,289.17,0.43,0.72,0.17,5
3,64085.9,2023_03_24_17_48_05,0,40.449079,-79.951965,9.83,0.9,289.18,0.43,0.72,0.17,7
4,64086.0,2023_03_24_17_48_06,0,40.449075,-79.951963,9.74,-0.9,289.18,0.43,0.72,0.17,9
...,...,...,...,...,...,...,...,...,...,...,...,...
3971,64487.0,2023_03_24_17_54_47,0,40.441458,-79.946384,2.55,8.1,287.03,0.32,0.48,0.14,13222
3972,64487.1,2023_03_24_17_54_47,0,40.441459,-79.946382,3.26,7.1,287.03,0.32,0.48,0.14,13229
3973,64487.2,2023_03_24_17_54_47,0,40.441459,-79.946381,2.98,-2.8,287.08,0.32,0.48,0.14,13233
3974,64487.3,2023_03_24_17_54_47,0,40.441460,-79.946380,3.11,1.3,287.10,0.32,0.48,0.15,13237


In [11]:
type([df["latitude"]][0][0])

numpy.float64

In [10]:
plot_trace(df)

In [ ]:
r1 = df[df["msg_num"] > 24297][df["msg_num"] < 28634]
r2 = df[df["msg_num"] > 30035][df["msg_num"] < 35190]
r3 = df[df["msg_num"] > 40949][df["msg_num"] < 43839]
r4 = df[df["msg_num"] > 57095][df["msg_num"] < 59583]

In [ ]:
layers = [{"below": "traces",
           "sourcetype": "raster",
           "source": ["https://imagery.pasda.psu.edu/arcgis/rest/services/pasda/PEMAImagery2018_2020/MapServer/WMTS/tile/1.0.0/pasda_PEMAImagery2018_2020/default/default028mm/{z}/{y}/{x}.png"]}]

fig = px.scatter_mapbox(prep_trace(r1),
                        lat="latitude", lon="longitude",
                        color="velocity",
                        hover_data={
                            "latitude": False,
                            "longitude": False,
                            "time": True,
                            "velocity": True,
                            "acceleration": True},
                        zoom=17, size_max=18,
                        height=800, width=1200)

fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=layers
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig.write_html(BASEDIR+"velotrace.html")



In [ ]:
layers = [{"below": "traces",
           "sourcetype": "raster",
           "source": ["https://imagery.pasda.psu.edu/arcgis/rest/services/pasda/PEMAImagery2018_2020/MapServer/WMTS/tile/1.0.0/pasda_PEMAImagery2018_2020/default/default028mm/{z}/{y}/{x}.png"]}]

d = pd.concat([prep_trace(r1, 1), prep_trace(r2, 2), prep_trace(r3, 3), prep_trace(r4, 4)])

fig = px.line_mapbox(d,
                        lat="latitude", lon="longitude",
                        color="roll_idx",
                        hover_data={
                            "latitude": False,
                            "longitude": False,
                            "time": True,
                            "velocity": True,
                            "acceleration": True},
                        zoom=17, #size_max=18,
                        height=800, width=1200)

fig.update_traces(line=dict(width=6))


fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=layers
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig.write_html(BASEDIR+"multiroll.html")



In [ ]:
layers = [{"below": "traces",
           "sourcetype": "raster",
           "source": ["https://imagery.pasda.psu.edu/arcgis/rest/services/pasda/PEMAImagery2018_2020/MapServer/WMTS/tile/1.0.0/pasda_PEMAImagery2018_2020/default/default028mm/{z}/{y}/{x}.png"]}]

fig = px.scatter_mapbox(prep_trace(r1),
                        lat="latitude", lon="longitude",
                        color="velocity",
                        hover_data={
                            "latitude": False,
                            "longitude": False,
                            "time": True,
                            "velocity": True,
                            "acceleration": True},
                        zoom=17, size_max=18,
                        height=800, width=1200)

fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=layers
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig.write_html(BASEDIR+"velotrace.html")



In [ ]:
q = prep_trace(r3)

fig = px.line(q, x="time", y=["velocity", "acceleration"], title="Roll 3 Vel/Acc Data")
fig.show()

fig.write_html(BASEDIR+"pusher2.html")

In [ ]:
q = prep_trace(r1, 1)
q["Velocity (Roll 1)"] = q["velocity"]
q["Acceleration (Roll 1)"] = q["acceleration"]
q2 = prep_trace(r3, 3)
q2["Velocity (Roll 3)"] = q2["velocity"]
q2["Acceleration (Roll 3)"] = q2["acceleration"]

fig = px.line(q, x="time", y=["Acceleration (Roll 1)"],
              title="Vel/Acc Data")
fig.update_traces(line_color='#00ff00')

fig.add_traces(
    list(px.line(q2, x="time", y=["Acceleration (Roll 3)"]).select_traces())
)
fig.show()

fig.write_html(BASEDIR+"pusher2.html")

In [ ]:
t = prep_trace(r1)
t = t.reset_index().to_dict(orient='list')
out = list(zip(t["timestamp"], t["latitude"], t["longitude"], t["velocity"], t["acceleration"]))
out = [list(x) for x in out]

with open("dict1.txt", "w+") as f:
    f.write(repr(out))


In [ ]:
plot_trace(r4)

In [ ]:
plt.plot(r1["timestamp"], r1["ground_speed"])
plt.plot(r1["timestamp"], BSpline(*tck1)(r1["timestamp"]))


In [ ]:
sns.lineplot(r1, x="timestamp", y="ground_speed")
sns.lineplot(r1, x="timestamp", y="acceleration")

In [ ]:
#df.to_csv(fn.replace(".ubx", "-lowaccuracy.csv"))
#fig.write_html(fn.replace(".ubx", "-lowaccuracy.html"))